# ⚽ Soccer Betting Predictor

This notebook demonstrates how to use the FootyStats API to predict soccer match outcomes for betting purposes using machine learning.

## 🎯 Features
- Real-time API integration with FootyStats
- Machine learning models (Random Forest & Gradient Boosting)
- Value betting analysis with Kelly Criterion
- Risk management recommendations

## 📦 Setup and Imports

In [1]:
# Import the predictor system
from enhanced_soccer_predictor import BettingPredictor, FootyStatsAPI

# Standard imports for data analysis
import numpy as np
import requests
import json
from datetime import datetime
import time

print("✅ Imports successful!")
print("📊 Ready to predict soccer matches!")

✅ Imports successful!
📊 Ready to predict soccer matches!


## 🔑 Configuration

Set up your API key and initialize the predictor system.

In [2]:
# Your FootyStats API key
API_KEY = "ceb338b9bcb82a452efc114fb2d3cccac67f58be1569e7b5acf1d2195adeae11"

# Initialize the predictor
predictor = BettingPredictor(API_KEY)

print("🚀 Predictor initialized successfully!")

🚀 Predictor initialized successfully!


## 🧪 Test API Connection

Let's verify that we can connect to the FootyStats API and fetch data.

In [3]:
# Test API connection
api = FootyStatsAPI(API_KEY)

print("🏈 Testing API Connection...")
print("=" * 40)

# Test Premier League data (ID: 1625)
matches = api.get_league_matches(1625)

if isinstance(matches, list) and len(matches) > 0:
    print(f"✅ Success! Found {len(matches)} Premier League matches")
    
    # Show sample match
    sample = matches[0]
    print(f"\n📊 Sample Match:")
    print(f"   🏠 Home: {sample.get('home_name', 'Unknown')}")
    print(f"   🚌 Away: {sample.get('away_name', 'Unknown')}")
    print(f"   💰 Odds - Home: {sample.get('odds_ft_1')}, Draw: {sample.get('odds_ft_x')}, Away: {sample.get('odds_ft_2')}")
    print(f"   ⚽ Score: {sample.get('homeGoalCount')} - {sample.get('awayGoalCount')}")
else:
    print("❌ API connection failed")
    print(f"Response: {matches}")

🏈 Testing API Connection...
✅ Success! Found 380 Premier League matches

📊 Sample Match:
   🏠 Home: Manchester United
   🚌 Away: Leicester City
   💰 Odds - Home: 1.37, Draw: 4.98, Away: 9.81
   ⚽ Score: 2 - 1


## 🤖 Train Machine Learning Models

Train the prediction models using historical match data, or load existing models if available.

In [4]:
MODEL_PATH = "soccer_prediction_models.pkl"

print("🧠 Setting up machine learning models...")
print("=" * 50)

# Try to load existing models
if not predictor.load_models(MODEL_PATH):
    print("📚 Training new models with Premier League data...")
    
    # Collect training data
    X, y, match_info = predictor.collect_training_data(league_id=1625, pages=3)
    
    print(f"📊 Training data: {len(X)} matches with {len(X[0])} features")
    print(f"🎯 Outcomes - Away: {np.sum(y==0)}, Draw: {np.sum(y==1)}, Home: {np.sum(y==2)}")
    
    # Train models
    predictor.train_models(X, y)
    
    # Save models
    predictor.save_models(MODEL_PATH)
    print("💾 Models saved successfully!")
else:
    print("✅ Loaded existing trained models!")

print("\n🎉 Models ready for predictions!")

INFO:enhanced_soccer_predictor:Models loaded from soccer_prediction_models.pkl


🧠 Setting up machine learning models...
✅ Loaded existing trained models!

🎉 Models ready for predictions!


## 🔮 Make Predictions

Now let's make predictions for some example matches using current betting odds.

In [5]:
# Example matches with current odds
test_matches = [
    {
        'name': 'Manchester City vs Arsenal',
        'home_odds': 1.8, 'draw_odds': 3.5, 'away_odds': 4.2
    },
    {
        'name': 'Liverpool vs Chelsea', 
        'home_odds': 2.1, 'draw_odds': 3.2, 'away_odds': 3.4
    },
    {
        'name': 'Brighton vs Everton',
        'home_odds': 2.3, 'draw_odds': 3.1, 'away_odds': 3.2
    },
    {
        'name': 'Tottenham vs Newcastle',
        'home_odds': 1.9, 'draw_odds': 3.4, 'away_odds': 4.0
    }
]

print("🔮 Making Match Predictions")
print("=" * 40)

for match in test_matches:
    print(f"\n🏆 {match['name']}")
    print(f"📊 Odds - Home: {match['home_odds']}, Draw: {match['draw_odds']}, Away: {match['away_odds']}")
    
    # Get predictions
    predictions = predictor.predict_match_outcome(
        match['home_odds'], match['draw_odds'], match['away_odds']
    )
    
    # Analyze betting value
    value_analysis = predictor.analyze_betting_value(
        predictions, match['home_odds'], match['draw_odds'], match['away_odds']
    )
    
    # Display results for each model
    for model_name, analysis in value_analysis.items():
        print(f"\n   🤖 {model_name.upper()}")
        print(f"      Prediction: {analysis['prediction']} (Confidence: {analysis['confidence']:.3f})")
        
        if analysis['value_bets']:
            print(f"      💰 Value Opportunities:")
            for bet in analysis['value_bets']:
                print(f"         {bet['outcome']}: {bet['value']:.1%} value, Kelly: {bet['kelly_fraction']:.1%} of bankroll")
                print(f"            Model prob: {bet['model_probability']:.3f}, Implied prob: {bet['implied_probability']:.3f}")
        else:
            print(f"      ❌ No value bets identified")

print("\n" + "="*50)
print("✅ Predictions complete!")

🔮 Making Match Predictions

🏆 Manchester City vs Arsenal
📊 Odds - Home: 1.8, Draw: 3.5, Away: 4.2

   🤖 RANDOM_FOREST
      Prediction: Home Win (Confidence: 0.430)
      ❌ No value bets identified

   🤖 GRADIENT_BOOSTING
      Prediction: Home Win (Confidence: 0.561)
      ❌ No value bets identified

🏆 Liverpool vs Chelsea
📊 Odds - Home: 2.1, Draw: 3.2, Away: 3.4

   🤖 RANDOM_FOREST
      Prediction: Away Win (Confidence: 0.350)
      💰 Value Opportunities:
         Away Win: 19.0% value, Kelly: 7.9% of bankroll
            Model prob: 0.350, Implied prob: 0.294

   🤖 GRADIENT_BOOSTING
      Prediction: Home Win (Confidence: 0.372)
      ❌ No value bets identified

🏆 Brighton vs Everton
📊 Odds - Home: 2.3, Draw: 3.1, Away: 3.2

   🤖 RANDOM_FOREST
      Prediction: Away Win (Confidence: 0.500)
      💰 Value Opportunities:
         Away Win: 60.0% value, Kelly: 25.0% of bankroll
            Model prob: 0.500, Implied prob: 0.312

   🤖 GRADIENT_BOOSTING
      Prediction: Away Win (Confid

## 🎯 Custom Match Prediction

Enter your own match odds to get predictions and value analysis.

In [6]:
# Customize these values for your match
custom_match = {
    'name': 'Your Custom Match',
    'home_odds': 2.5,   # Change this to actual home win odds
    'draw_odds': 3.0,   # Change this to actual draw odds
    'away_odds': 2.8    # Change this to actual away win odds
}

print(f"🎯 Custom Prediction: {custom_match['name']}")
print(f"📊 Odds - Home: {custom_match['home_odds']}, Draw: {custom_match['draw_odds']}, Away: {custom_match['away_odds']}")

# Get prediction
predictions = predictor.predict_match_outcome(
    custom_match['home_odds'], custom_match['draw_odds'], custom_match['away_odds']
)

# Show detailed probabilities
print("\n📊 Model Probabilities:")
for model_name, pred_data in predictions.items():
    print(f"\n🤖 {model_name.upper()}:")
    print(f"   Predicted outcome: {pred_data['prediction']} (confidence: {pred_data['confidence']:.3f})")
    print("   Probabilities:")
    for outcome, prob in pred_data['probabilities'].items():
        print(f"     {outcome.replace('_', ' ').title()}: {prob:.3f} ({prob*100:.1f}%)")

# Value analysis
value_analysis = predictor.analyze_betting_value(
    predictions, custom_match['home_odds'], custom_match['draw_odds'], custom_match['away_odds']
)

print("\n💰 Betting Recommendations:")
any_value_found = False

for model_name, analysis in value_analysis.items():
    if analysis['value_bets']:
        any_value_found = True
        print(f"\n🎯 {model_name.upper()} suggests:")
        for bet in analysis['value_bets']:
            print(f"   💎 {bet['outcome']}: {bet['value']:.1%} edge")
            print(f"      Recommended bet: {bet['kelly_fraction']:.1%} of bankroll")
            print(f"      Odds: {bet['odds']}, Expected value: positive")

if not any_value_found:
    print("❌ No value betting opportunities identified")
    print("💡 Suggestion: Wait for better odds or consider different matches")

🎯 Custom Prediction: Your Custom Match
📊 Odds - Home: 2.5, Draw: 3.0, Away: 2.8

📊 Model Probabilities:

🤖 RANDOM_FOREST:
   Predicted outcome: Away Win (confidence: 0.500)
   Probabilities:
     Away Win: 0.500 (50.0%)
     Draw: 0.200 (20.0%)
     Home Win: 0.300 (30.0%)

🤖 GRADIENT_BOOSTING:
   Predicted outcome: Home Win (confidence: 0.375)
   Probabilities:
     Away Win: 0.356 (35.6%)
     Draw: 0.270 (27.0%)
     Home Win: 0.375 (37.5%)

💰 Betting Recommendations:

🎯 RANDOM_FOREST suggests:
   💎 Away Win: 40.0% edge
      Recommended bet: 22.2% of bankroll
      Odds: 2.8, Expected value: positive


## 📈 Analyze Recent Match Data

Let's examine some recent matches and see how our model would have performed.

In [7]:
# Fetch recent matches
recent_matches = api.get_league_matches(1625)  # Premier League

print("📈 Recent Match Analysis")
print("=" * 30)

if recent_matches:
    # Analyze first 5 matches
    for i, match in enumerate(recent_matches[:5]):
        if match.get('odds_ft_1') and match.get('homeGoalCount') is not None:
            print(f"\n🏆 Match {i+1}: {match.get('home_name', 'Team A')} vs {match.get('away_name', 'Team B')}")
            print(f"   📊 Odds: {match.get('odds_ft_1'):.2f} / {match.get('odds_ft_x'):.2f} / {match.get('odds_ft_2'):.2f}")
            print(f"   ⚽ Result: {match.get('homeGoalCount')} - {match.get('awayGoalCount')}")
            
            # Determine actual outcome
            home_goals = match.get('homeGoalCount', 0)
            away_goals = match.get('awayGoalCount', 0)
            
            if home_goals > away_goals:
                actual = "Home Win"
            elif away_goals > home_goals:
                actual = "Away Win"
            else:
                actual = "Draw"
                
            print(f"   🎯 Actual outcome: {actual}")
            
            # Get our model's prediction
            try:
                prediction = predictor.predict_match_outcome(
                    float(match.get('odds_ft_1', 2.0)),
                    float(match.get('odds_ft_x', 3.0)),
                    float(match.get('odds_ft_2', 2.0))
                )
                
                # Show model prediction vs actual
                for model_name, pred_data in prediction.items():
                    model_pred = pred_data['prediction']
                    confidence = pred_data['confidence']
                    
                    if model_pred == actual:
                        result_icon = "✅"
                    else:
                        result_icon = "❌"
                        
                    print(f"   🤖 {model_name}: {model_pred} ({confidence:.3f}) {result_icon}")
                    
            except Exception as e:
                print(f"   ⚠️ Could not analyze: {e}")
                
else:
    print("❌ No recent match data available")

print("\n" + "="*40)
print("📊 Analysis complete!")

📈 Recent Match Analysis

🏆 Match 1: Manchester United vs Leicester City
   📊 Odds: 1.37 / 4.98 / 9.81
   ⚽ Result: 2 - 1
   🎯 Actual outcome: Home Win
   🤖 random_forest: Home Win (0.580) ✅
   🤖 gradient_boosting: Home Win (0.623) ✅

🏆 Match 2: Newcastle United vs Tottenham Hotspur
   📊 Odds: 4.51 / 3.77 / 1.84
   ⚽ Result: 1 - 2
   🎯 Actual outcome: Away Win
   🤖 random_forest: Away Win (0.610) ✅
   🤖 gradient_boosting: Home Win (0.565) ❌

🏆 Match 3: AFC Bournemouth vs Cardiff City
   📊 Odds: 2.03 / 3.51 / 3.96
   ⚽ Result: 2 - 0
   🎯 Actual outcome: Home Win
   🤖 random_forest: Draw (0.400) ❌
   🤖 gradient_boosting: Draw (0.462) ❌

🏆 Match 4: Fulham vs Crystal Palace
   📊 Odds: 2.31 / 3.44 / 3.26
   ⚽ Result: 0 - 2
   🎯 Actual outcome: Away Win
   🤖 random_forest: Away Win (0.420) ✅
   🤖 gradient_boosting: Home Win (0.376) ❌

🏆 Match 5: Huddersfield Town vs Chelsea
   📊 Odds: 7.47 / 4.27 / 1.51
   ⚽ Result: 0 - 3
   🎯 Actual outcome: Away Win
   🤖 random_forest: Away Win (0.700) ✅
  

## 💰 Betting Strategy Helper

Calculate optimal bet sizes and potential returns using the Kelly Criterion.

In [8]:
def calculate_betting_returns(odds, probability, bankroll, kelly_fraction):
    """Calculate potential betting outcomes"""
    bet_amount = bankroll * kelly_fraction
    
    if bet_amount > 0:
        potential_profit = bet_amount * (odds - 1)
        potential_loss = bet_amount
        expected_value = (probability * potential_profit) - ((1 - probability) * potential_loss)
        
        return {
            'bet_amount': bet_amount,
            'potential_profit': potential_profit,
            'potential_loss': potential_loss,
            'expected_value': expected_value,
            'roi': (expected_value / bet_amount * 100) if bet_amount > 0 else 0
        }
    return None

# Example betting scenario
print("💰 Betting Strategy Calculator")
print("=" * 35)

# Set your bankroll
bankroll = 1000  # Change this to your actual bankroll
print(f"💳 Bankroll: ${bankroll:,.2f}")

# Example match for betting analysis
example_match = {
    'name': 'Example Match Analysis',
    'home_odds': 2.2,
    'draw_odds': 3.1, 
    'away_odds': 3.5
}

print(f"\n🏆 {example_match['name']}")
predictions = predictor.predict_match_outcome(
    example_match['home_odds'], example_match['draw_odds'], example_match['away_odds']
)

value_analysis = predictor.analyze_betting_value(
    predictions, example_match['home_odds'], example_match['draw_odds'], example_match['away_odds']
)

print("\n📊 Betting Analysis:")
for model_name, analysis in value_analysis.items():
    print(f"\n🤖 {model_name.upper()}:")
    
    if analysis['value_bets']:
        for bet in analysis['value_bets']:
            print(f"\n   💎 {bet['outcome']} Opportunity:")
            print(f"      Odds: {bet['odds']:.2f}")
            print(f"      Model Probability: {bet['model_probability']:.3f} ({bet['model_probability']*100:.1f}%)")
            print(f"      Market Probability: {bet['implied_probability']:.3f} ({bet['implied_probability']*100:.1f}%)")
            print(f"      Edge: {bet['value']:.1%}")
            print(f"      Kelly Recommendation: {bet['kelly_fraction']:.1%} of bankroll")
            
            # Calculate returns
            returns = calculate_betting_returns(
                bet['odds'], bet['model_probability'], bankroll, bet['kelly_fraction']
            )
            
            if returns:
                print(f"\n   💵 Betting Calculation:")
                print(f"      Recommended bet: ${returns['bet_amount']:.2f}")
                print(f"      Potential profit: ${returns['potential_profit']:.2f}")
                print(f"      Potential loss: ${returns['potential_loss']:.2f}")
                print(f"      Expected value: ${returns['expected_value']:.2f}")
                print(f"      Expected ROI: {returns['roi']:.2f}%")
    else:
        print("   ❌ No value opportunities found")

print("\n" + "="*50)
print("⚠️  Important Reminders:")
print("   • Only bet what you can afford to lose")
print("   • Past performance doesn't guarantee future results")
print("   • Consider external factors (injuries, team news, etc.)")
print("   • Use proper bankroll management")
print("   • This is for educational purposes only")

💰 Betting Strategy Calculator
💳 Bankroll: $1,000.00

🏆 Example Match Analysis

📊 Betting Analysis:

🤖 RANDOM_FOREST:

   💎 Away Win Opportunity:
      Odds: 3.50
      Model Probability: 0.400 (40.0%)
      Market Probability: 0.286 (28.6%)
      Edge: 40.0%
      Kelly Recommendation: 16.0% of bankroll

   💵 Betting Calculation:
      Recommended bet: $160.00
      Potential profit: $400.00
      Potential loss: $160.00
      Expected value: $64.00
      Expected ROI: 40.00%

🤖 GRADIENT_BOOSTING:
   ❌ No value opportunities found

⚠️  Important Reminders:
   • Only bet what you can afford to lose
   • Past performance doesn't guarantee future results
   • Consider external factors (injuries, team news, etc.)
   • Use proper bankroll management
   • This is for educational purposes only


## 🎉 Next Steps

Now that you've learned how to use the soccer betting predictor:

### 🚀 What You Can Do:
1. **Modify the code cells above** to analyze different matches
2. **Change the odds values** in the custom prediction section
3. **Adjust your bankroll** in the betting calculator
4. **Experiment with different league IDs** (if your API key supports them)
5. **Save interesting predictions** by running the cells

### 📈 Advanced Usage:
- Collect data from multiple leagues
- Build ensemble predictions using multiple models
- Track prediction accuracy over time
- Implement more sophisticated betting strategies

### ⚠️ Remember:
- **Always gamble responsibly**
- **Only bet what you can afford to lose**
- **Consider this a learning tool, not a guarantee**
- **Do your own research on teams, injuries, and form**

---

**Happy predicting! 🎯⚽💰**